In [1]:
#!/usr/bin/env python3
import numpy as np
import transformations as tf

/home/ajinkya/Software/ajinkya/active_champ/plots/transformations.py:1928: UserWarning: No module named '_transformations'
  warnings.warn(str(err))


In [2]:
def find_axis_in_camera_frame(pt, ref):
    quat = ref[3:]
    t = tf.quaternion_matrix(quat)
    pt = np.append(pt, 1.)
    new_pt = t.dot(pt)[:3]
    return new_pt

In [3]:
def find_axis_in_global_frame(pt):
    quat = np.array([ 0., -0.707, 0., 0.707])
    t = tf.quaternion_matrix(quat)
    pt = np.append(pt, 1.)
    new_pt = t.dot(pt)[:3]
    return new_pt

In [4]:
def find_axis_in_global_frame_from_quat(quat_in):
    t1 = tf.quaternion_matrix(quat_in)
    x = np.array([0., 0., 1. ,1.])
    pt = t1.dot(x)[:3]
    new_pt = find_axis_in_global_frame(pt)
    return new_pt

In [5]:
def calculate_error(vec, ax_mat):
    abs_err = []
    for ax in ax_mat:
        abs_err.append(1. - abs((np.dot(ax.T, vec))/(np.linalg.norm(ax)* np.linalg.norm(vec))))
    return np.array(abs_err)

In [6]:
## Axis are wrt to some origin. ZWe need to convert thtm wrt to a common frame for comparison. 
## We can convert them to a transform wrt Camera frame using the rigid body parms we know

champ_center = np.reshape([0.408347462728,
-0.43070113035,
-0.110778859282], (1,3))


champ_axes = np.reshape([-0.582976265224,
-0.472297565236,
0.638679360885,
-0.170769897922], (1,4))

dummy = np.zeros((champ_axes.shape[0], champ_axes.shape[1]))
dummy[:,1:] = champ_axes[:, :-1]
dummy[:,0] = champ_axes[:, -1]
champ_axes = dummy 

champ_radius = np.array([0.505937440872])

champ_center_global = []
for pt in champ_center:
    pt_in_global =  find_axis_in_global_frame(pt)
    champ_center_global.append(pt_in_global)

champ_axes_global = []
for axis in champ_axes:
    axis_in_global =  find_axis_in_global_frame_from_quat((axis))
    champ_axes_global.append(axis_in_global)

In [7]:
gt_axis = np.array([0., 0., 1.])
gt_rad = 0.45
gt_center = np.array([-0.07834291, -0.3022653 , -0.02670656])
abs_err = calculate_error(gt_axis, champ_axes_global)

print("\nCHAMP Absolute Error in Axis:")
mean_abs_err = np.mean(abs_err, axis=0)
print("Mean: ", mean_abs_err)
std_abs_err = np.std(abs_err, axis=0)
print("std: ", std_abs_err)

abs_err_rad = abs(abs(champ_radius) - gt_rad)
print("\nCHAMP Absolute Error in Radius:")
mean_abs_err_rad = np.mean(abs_err_rad, axis=0)
print("Mean: ", mean_abs_err_rad)
std_abs_err_rad = np.std(abs_err_rad, axis=0)
print("std: ", std_abs_err_rad)

abs_err_center = []
for p in champ_center:
    abs_err_center.append(np.linalg.norm(gt_center - p))
print("\nCHAMP Absolute Error in Center:")
mean_abs_err_center = np.mean(abs_err_center, axis=0)
print("Mean: ", mean_abs_err_center)
std_abs_err_center = np.std(abs_err_center, axis=0)
print("std: ", std_abs_err_center)


CHAMP Absolute Error in Axis:
Mean:  0.4166385970390426
std:  0.0

CHAMP Absolute Error in Radius:
Mean:  0.05593744087199998
std:  0.0

CHAMP Absolute Error in Center:
Mean:  0.5103248307993408
std:  0.0


In [8]:
abs_err_center

[0.5103248307993408]

In [12]:
## Act-CHAMP
actCHAMP_center = np.reshape([0.408347462728,
-0.43070113035,
-0.110778859282,
0.325690033844,
-0.0811431453607,
0.0220380779832,
-0.216870308766,
-0.283168593619,
0.103517620447,
0.177080603674,
-0.268679376611,
0.0155597397094,
0.0977043970728,
-0.245151228683,
0.0328268894163,
-0.385044849375,
-0.23345228468,
0.124028131214,], (6,3))


actCHAMP_axes = np.reshape([-0.582976265224,
-0.472297565236,
0.638679360885,
-0.170769897922,
0.432517486408,
0.700301606125,
-0.403931886633,
0.399180805364,
-0.432326520169,
0.363487946111,
0.734317687077,
0.376494126696,
0.459226718467,
-0.60272228652,
0.219405874746,
0.614571174483,
0.670454467941,
0.0912854778782,
0.566939952187,
-0.469826413221,
-0.428250392174,
-0.426155380831,
-0.496719810647,
0.623107232106,], (6,4))

dummy = np.zeros((actCHAMP_axes.shape[0], actCHAMP_axes.shape[1]))
dummy[:,1:] = actCHAMP_axes[:, :-1]
dummy[:,0] = actCHAMP_axes[:, -1]
actCHAMP_axes = dummy 

actCHAMP_radius = np.array([0.505937440872,
0.328940337334,
0.261999519169,
0.198789894215,
0.123391619752,
0.41029539234,])

actCHAMP_axes_global = []
for axis in actCHAMP_axes:
    actCHAMP_axis =  find_axis_in_global_frame_from_quat(axis)
    actCHAMP_axes_global.append(actCHAMP_axis)

actCHAMP_abs_err = calculate_error(gt_axis, actCHAMP_axes_global)
print("\nActCHAMP Absolute Error in Axis:")
actCHAMP_mean_abs_err = np.mean(actCHAMP_abs_err, axis=0)
print("Mean: ", actCHAMP_mean_abs_err)
actCHAMP_std_abs_err = np.std(actCHAMP_abs_err, axis=0)
print("std: ", actCHAMP_std_abs_err)

actCHAMP_abs_err_rad = abs(actCHAMP_radius - gt_rad)
print("\nActCHAMP Absolute Error in Radius:")
actCHAMP_mean_abs_err_rad = np.mean(actCHAMP_abs_err_rad, axis=0)
print("Mean: ", actCHAMP_mean_abs_err_rad)
actCHAMP_std_abs_err_rad = np.std(actCHAMP_abs_err_rad, axis=0)
print("std: ", actCHAMP_std_abs_err_rad)


actCHAMP_abs_err_center = []
for p in actCHAMP_center:
    actCHAMP_abs_err_center.append(np.linalg.norm( p - gt_center))

print("\nActCHAMP Absolute Error in center:")
actCHAMP_mean_abs_err_center = np.mean(actCHAMP_abs_err_center, axis=0)
print("Mean: ", actCHAMP_mean_abs_err_center)
actCHAMP_std_abs_err_center = np.std(actCHAMP_abs_err_center, axis=0)
print("std: ", actCHAMP_std_abs_err_center)



ActCHAMP Absolute Error in Axis:
Mean:  0.5877227216942659
std:  0.20486414294231414

ActCHAMP Absolute Error in Radius:
Mean:  0.16375344634366665
std:  0.10295102398584183

ActCHAMP Absolute Error in center:
Mean:  0.32810842874507024
std:  0.12450160756556337


In [10]:
np.shape(champ_axes[:,:3])

(1, 3)

In [ ]:
## CHANGEPOINT Analysis
champ_cp = np.reshape([-0.00193311273984,
0.142402253709,
-0.00848558509461,
-0.00193311273984,
0.142402253709,
-0.00848558509461,
-0.000867558914176,
0.141855125978,
-0.0049918187279,
-0.00193311273984,
0.142402253709,
-0.00848558509461,
-0.00193311273984,
0.142402253709,
-0.00848558509461,
-0.00193311273984,
0.142402253709,
-0.00848558509461,
-0.00193311273984,
0.142402253709,
-0.00848558509461,
-0.001933,
0.142402,
-0.008486,
-0.00193311273984,
0.142402253709,
-0.00848558509461,
-0.00193311273984,
0.142402253709,
-0.00848558509461,], (10, 3))

ActCHAMP_cp = np.reshape([-0.00193311273984,
0.142402253709,
-0.00848558509461,
-0.000867558914176,
0.141855125978,
-0.0049918187279,
-0.00193311273984,
0.142402253709,
-0.00848558509461,
-0.00193311273984,
0.142402253709,
-0.00848558509461,
-0.00193311273984,
0.142402253709,
-0.00848558509461,
-0.000867558914176,
0.141855125978,
-0.0049918187279,
-0.000867558914176,
0.141855125978,
-0.0049918187279,
-0.00193311273984,
0.142402253709,
-0.00848558509461,
-0.00193311273984,
0.142402253709,
-0.00848558509461,
-0.00193311273984,
0.142402253709,
-0.00848558509461,], (10, 3))

# gt_cp = np.mean(np.concatenate((champ_cp, ActCHAMP_cp), axis=0), axis=0)
# print("gt_cp: ", gt_cp)

gt_cp = np.array([-0.00172, 0.14229282, -0.00778685])

print("Error in CHAMP cp")
champ_cp_err = []
for p in champ_cp:
    champ_cp_err.append(np.linalg.norm(p - gt_cp))

mean_champ_cp_err = np.mean(champ_cp_err)
print("Mean err in CHAMP cp: ", mean_champ_cp_err)
std_champ_cp_err = np.std(champ_cp_err)
print("std in err in CHAMP cp: ", std_champ_cp_err)

print("\n\nError in ActCHAMP cp")
ActCHAMP_cp_err = []
for p in ActCHAMP_cp:
    ActCHAMP_cp_err.append(np.linalg.norm(p - gt_cp))

mean_ActCHAMP_cp_err = np.mean(ActCHAMP_cp_err)
print("Mean err in ActCHAMP cp: ", mean_ActCHAMP_cp_err)
std_ActCHAMP_cp_err = np.std(ActCHAMP_cp_err)
print("std in err in ActCHAMP cp: ", std_ActCHAMP_cp_err)